# Use examples of [edges](https://github.com/romainsacchi/edges)

Author: [romainsacchi](https://github.com/romainsacchi)

This notebook shows examples on how to use `edge` to use location-specific
characterization factors in the characterization matrix of `bw2calc`.

## Requirements

* **Pyhton 3.10 or higher (up to 3.11) is highly recommended**
* `bw2calc >= 2.0.0`

# Use case with [brightway2](https://brightway.dev/)

`brightway2` is an open source LCA framework for Python.
To use `premise` from `brightway2`, it requires that you have an activated `brightway2` project with a `biosphere3` database as well as an [ecoinvent](https://ecoinvent.prg) v.3 cut-off or consequential database registered in that project. Please refer to the brightway [documentation](https://brightway.dev) if you do not know how to create a project and install ecoinvent.

In [1]:
from edges import SpatialLCA, get_available_methods
import bw2data

### List of available methods


In [2]:
get_available_methods()

[('AWARE 1.2c', 'Country', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'irri', 'may'),
 ('AWARE 1.2c', 'Country', 'irri', 'nov'),
 ('AWARE 1.2c', 'Country', 'irri', 'oct'),
 ('AWARE 1.2c', 'Country', 'irri', 'sep'),
 ('AWARE 1.2c', 'Country', 'irri', 'yearly'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'apr'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'aug'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'dec'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'feb'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jan'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jul'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'jun'),
 ('AWARE 1.2c', 'Country', 'non', 'irri', 'mar'),
 ('AWARE 1.2c', 'Country', 'non', 'irr

### Spatialized LCIA

The `SpatialLCA` class is a subclass of `bw2calc.LCA` that allows for the use of location-specific characterization factors in the characterization matrix of `bw2calc`. `SpatialLCA`can therefore be used as a drop-in replacement for `bw2calc.LCA` in most cases.

In [3]:
# activate the bw project
bw2data.projects.set_current("bw25_ei310")

In [4]:
act = bw2data.Database("ecoinvent-3.10-cutoff").random()
act

'electricity, high voltage, import from US-MRO' (kilowatt hour, CA-SK, None)

In [5]:
method = ('AWARE 1.2c', 'Country', 'unspecified', 'yearly')

In [6]:
LCA = SpatialLCA({act: 1}, method)
LCA.lci()
LCA.lcia()
LCA.score

/Users/romain/anaconda3/envs/bw25/lib/python3.11/site-packages/scikits/umfpack/umfpack.py:736: UmfpackWarning: (almost) singular matrix! (estimated cond. number: 1.21e+13)
  warnings.warn(msg, UmfpackWarning)


0.10860511854635746

In [ ]:
# let's check locations that have been ignored, if any
LCA.ignored_locations

### Generate dataframe of characterization factors used

In [8]:
df = LCA.generate_cf_table()

In [10]:
# we can see under the "CF" column
# the characterization factors used for each exchange in the system
df.head()

,flow,name,reference product,location,amount,CF,impact
0,"(Water, cooling, unspecified natural origin, (...",cast iron production,cast iron,RoW,6.288335e-07,28.180497,1.772084e-05
1,"(Water, cooling, unspecified natural origin, (...","polystyrene production, high impact","polystyrene, high impact",RER,1.138145e-09,17.328016,1.972179e-08
2,"(Water, cooling, unspecified natural origin, (...","cast iron milling, average","cast iron removed by milling, average",RoW,1.305994e-13,28.180497,3.680357e-12
3,"(Water, cooling, unspecified natural origin, (...","electricity production, oil","electricity, high voltage",CA-PE,1.992256e-10,1.713382,3.413496e-10
4,"(Water, cooling, unspecified natural origin, (...","polypropylene production, granulate","polypropylene, granulate",RER,7.043196e-09,17.328016,1.220446e-07
